In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
torch.hub.set_dir('/mnt/DONNEES/hbrachemi/.cache/torch/hub/checkpoints/')

In [4]:
from victimModel import *

In [5]:
model_ft= VictimModel("vgg16",True,10)

In [6]:
a=model_ft.evaluate(testloader)

100%|██████████| 2500/2500 [00:10<00:00, 228.60it/s]


In [11]:
np.mean(a["energy"]["avg_case_cons"])/1e12

0.269705396224

In [12]:
np.mean(a["energy"]["ratio_cons"])

0.96944124

In [13]:
a['energy'].keys()

dict_keys(['avg_case_cons', 'worst_case_cons', 'ratio_cons'])